# MicroStrategy Embedding - How to embed an interactive Dossier during your analysis within Jupyter Notebook

### Python, JavaScript and HTML with Jupyter magic cells

Scenario: 

To demonstrate how to use a python Jupyter Notebook to navigate and find the Dossier we are interested in, display it inside Jupyter Notebook for interactive data analysis. This will help us better understand the data as published by business and before we start loading the data from the cube(s) into DataFrames. This way we will get better insight into the type of analysis done by the business, and how they prefer to see and digest their data to help us gain a better intuition about the business use case(s) and driver.

In this example I will show you how to use pure code all within Jupyter Notebook to accomplish this without having to go to MicroStrategy Web or needing to right-click to get object properties to obtain IDs for the project or document.  

This tutorial follows the **MicroStratety REST API with Python** examples here https://github.com/tatwan/MicroStratety-Python-REST-API and my previous blog on **How to Embed MicroStrategy Dossier with Jupyter Notebook** here http://www.tarekatwan.com/index.php/2018/01/how-to-embed-microstrategy-dossier-with-jupyter-notebook/  

### We start by loading the Python libraries

In [ ]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd

### Define our parameters 
`username`. `password`, and `baseURL`

In [4]:
### Parameters ###
username = 'Administrator'
password = ''
baseURL = "http://yourMstrEnv/MicroStrategyLibrary/api/"

Next we load our python functions from this tutorial https://github.com/tatwan/MicroStratety-Python-REST-API . All that was done is placing the python code/script into a `mstr.py` file then we use `import mstr`. You can name the file anything you want.

In [5]:
import mstr 

### Steps:
1. First we authenticate to get `authToken` and `sessionId` using the `login()` function 
2. After we are authenticated, we will list all the projects that we have access to using our `listProjects()` function
3. Select the project you want and save the `id` of that project into variable we will name `projectId`  
4. Next we list our library using the `getLibrary()` function in order to find the `id` of the published Dossier we are interested in. In this case we need the `id` from the `target` column and we will save it into `libraryId` variable. To get the `target` columns we pass `DEFAULT` for the flag parameter. If we use `FILTER_TOC` we don't get that column.

In [6]:
#step 1 - authenticate 

authToken, sessionId = mstr.login(baseURL, username, password)

Token: 4gbumu5r3fa49hiueniabur9vb
Session ID: {'JSESSIONID': '63DDC2C2CAD119E0FC8F82EEA3C6AFE4'}


In [7]:
#step 2 - search  projects

projectList = mstr.listProjects(baseURL, authToken, sessionId)
projectList

,id,name,description,status
0,B19DEDCC11D4E0EFC000EB9495D0F44F,MicroStrategy Tutorial,MicroStrategy Tutorial project and application...,0
1,AF09B3E3458F78B4FBE4DEB68528BF7B,Human Resources Analysis Module,The Human Resources Analysis Module analyses w...,0
2,4DD3B04B40D227471401609D630C76ED,Enterprise Manager,,0


In [8]:
#step 3 - Get the project ID of the project we are interested in 
# Tuorial Project (first row)

projectId = projectList.iloc[0][0]
projectId

'B19DEDCC11D4E0EFC000EB9495D0F44F'

In [9]:
# step 4 - Get the library List
# Here we see we have THREE published dossiers

libraryList = mstr.getLibrary(baseURL, authToken, sessionId, 'DEFAULT')
libraryList

,id,name,projectId,active,lastViewedTime,target
0,1B979449411E30E4E4502F918158EA40,Category Analysis,B19DEDCC11D4E0EFC000EB9495D0F44F,True,2018-08-11T07:49:40.000+0000,512EDAA1487128DBBCA43E8525E10A11
1,21A521BA4DB47ADAEBE19E9E9F7EC7D9,Executive Business User Data Dossier,B19DEDCC11D4E0EFC000EB9495D0F44F,True,2018-08-10T19:36:00.000+0000,FC6E8B6F4950540FC3595093E0FBA306
2,80AFEAD447DE2430F7E41FBB1B1EFCBA,Category Breakdown Dossier,B19DEDCC11D4E0EFC000EB9495D0F44F,True,2018-08-10T21:36:32.000+0000,95005DFF4C4829CF5EE6E98877726566


In [10]:
#Get the Target ID for the published Dossier we want to embed
# In this case I want 'Categry Analysis' first row, and I want the fifth column 'Target' id

libraryId = libraryList.iloc[0][5]
libraryId

'512EDAA1487128DBBCA43E8525E10A11'

Using iPython magic cell we use `%%html` to specifiy were we want to display our Dossier. Here we load the `embeddinglib.js` JavaScript library provided by MicroStrategy. Then we add our `<script></script>` which includes our javaccript code. You can also separate the javascript in a separate cell using the `%%javascript` magic cell as well.

Dossier will be displayed right below this `%%html` cell

In [12]:
%%html

<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
<script type="text/javascript" src="http://yourMstrEnv/MicroStrategyLibrary/javascript/embeddinglib.js"></script>
<h2> Embedding an Interactive Dossier </h2>
<div id="dossier1"></div>

<script>
//NOTE: we use our projectID and libraryId values and use them in the javaScript variables below

var project = 'B19DEDCC11D4E0EFC000EB9495D0F44F'
var library = '512EDAA1487128DBBCA43E8525E10A11'

var container = document.getElementById("dossier1"),
 
    url = "http://yourMstrEnv/MicroStrategyLibrary/app/" + project + '/' + library;
 
    microstrategy.dossier.create({
 
          url: url,
 
          enableResponsive: true,
 
          placeholder: container
       })
    
</script>